In [2]:
import xml.etree.ElementTree as ET
import pickle
import os
from os import listdir, getcwd
from os.path import join
import random
from shutil import copyfile
names={
 '6': '大螟',
 '7':'二化螟',
 '8': '稻纵卷叶螟',
 '9': '白背飞虱',
 '10': '褐飞虱属',
 '25': '地老虎',
 '41': '蝼蛄',
 '105': '粘虫',
 '110': '草地螟',
 '115': '甜菜夜蛾',
 '148': '黄足猎蝽',
 '156': '八点灰灯蛾',
 '222': '棉铃虫',
 '228': '二点委夜蛾',
 '235': '甘蓝夜蛾',
 '256': '蟋蟀',
 '280': '黄毒蛾',
 '310': '稻螟蛉',
 '387': '紫条尺蛾',
 '392': '水螟蛾',
 '394': '线委夜蛾',
 '398': '甜菜白带野螟',
 '401': '歧角螟',
 '402': '瓜绢野螟',
 '430': '豆野螟',
 '480': '石蛾',
 '485': '大黑鳃金龟',
 '673': '干纹冬夜蛾'}
classes = list(names.keys())
#classes=["ball"]
 
TRAIN_RATIO = 80
 
def clear_hidden_files(path):
    dir_list = os.listdir(path)
    for i in dir_list:
        abspath = os.path.join(os.path.abspath(path), i)
        if os.path.isfile(abspath):
            if i.startswith("._"):
                os.remove(abspath)
        else:
            clear_hidden_files(abspath)
def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)
 
def convert_annotation(image_id):
    # in_file = open('VOCdevkit/VOC2007/Annotations/%s.xml' %image_id)
    
    in_file = open(r'C:\Users\awsless\Desktop\VOCdevkit/VOC2007/Annotations/%s.xml' %image_id)
    out_file = open('VOCdevkit/VOC2007/YOLOLabels/%s.txt' %image_id, 'w')
    tree=ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)
 
    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult) == 1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')
    in_file.close()
    out_file.close()
wd = os.getcwd()
wd = os.getcwd()
data_base_dir = os.path.join(wd, "VOCdevkit/")
if not os.path.isdir(data_base_dir):
    os.mkdir(data_base_dir)
work_sapce_dir = os.path.join(data_base_dir, "VOC2007/")
if not os.path.isdir(work_sapce_dir):
    os.mkdir(work_sapce_dir)
annotation_dir = os.path.join(work_sapce_dir, "Annotations/")
if not os.path.isdir(annotation_dir):
        os.mkdir(annotation_dir)
clear_hidden_files(annotation_dir)
image_dir = os.path.join(work_sapce_dir, "JPEGImages/")
if not os.path.isdir(image_dir):
        os.mkdir(image_dir)
clear_hidden_files(image_dir)
yolo_labels_dir = os.path.join(work_sapce_dir, "YOLOLabels/")
if not os.path.isdir(yolo_labels_dir):
        os.mkdir(yolo_labels_dir)
clear_hidden_files(yolo_labels_dir)
yolov5_images_dir = os.path.join(data_base_dir, "images/")
if not os.path.isdir(yolov5_images_dir):
        os.mkdir(yolov5_images_dir)
clear_hidden_files(yolov5_images_dir)
yolov5_labels_dir = os.path.join(data_base_dir, "labels/")
if not os.path.isdir(yolov5_labels_dir):
        os.mkdir(yolov5_labels_dir)
clear_hidden_files(yolov5_labels_dir)
yolov5_images_train_dir = os.path.join(yolov5_images_dir, "train/")
if not os.path.isdir(yolov5_images_train_dir):
        os.mkdir(yolov5_images_train_dir)
clear_hidden_files(yolov5_images_train_dir)
yolov5_images_test_dir = os.path.join(yolov5_images_dir, "val/")
if not os.path.isdir(yolov5_images_test_dir):
        os.mkdir(yolov5_images_test_dir)
clear_hidden_files(yolov5_images_test_dir)
yolov5_labels_train_dir = os.path.join(yolov5_labels_dir, "train/")
if not os.path.isdir(yolov5_labels_train_dir):
        os.mkdir(yolov5_labels_train_dir)
clear_hidden_files(yolov5_labels_train_dir)
yolov5_labels_test_dir = os.path.join(yolov5_labels_dir, "val/")
if not os.path.isdir(yolov5_labels_test_dir):
        os.mkdir(yolov5_labels_test_dir)
clear_hidden_files(yolov5_labels_test_dir)
 
train_file = open(os.path.join(wd, "yolov5_train.txt"), 'w')
test_file = open(os.path.join(wd, "yolov5_val.txt"), 'w')
train_file.close()
test_file.close()
train_file = open(os.path.join(wd, "yolov5_train.txt"), 'a')
test_file = open(os.path.join(wd, "yolov5_val.txt"), 'a')
list_imgs = os.listdir(image_dir) # list image files
prob = random.randint(1, 100)
print("Probability: %d" % prob)
for i in range(0,len(list_imgs)):
    path = os.path.join(image_dir,list_imgs[i])
    if os.path.isfile(path):
        image_path = image_dir + list_imgs[i]
        voc_path = list_imgs[i]
        (nameWithoutExtention, extention) = os.path.splitext(os.path.basename(image_path))
        (voc_nameWithoutExtention, voc_extention) = os.path.splitext(os.path.basename(voc_path))
        annotation_name = nameWithoutExtention + '.xml'
        annotation_path = os.path.join(annotation_dir, annotation_name)
        label_name = nameWithoutExtention + '.txt'
        label_path = os.path.join(yolo_labels_dir, label_name)
    prob = random.randint(1, 100)
    print("Probability: %d" % prob)
    if(prob < TRAIN_RATIO): # train dataset
        if os.path.exists(annotation_path):
            train_file.write(image_path + '\n')
            convert_annotation(nameWithoutExtention) # convert label
            copyfile(image_path, yolov5_images_train_dir + voc_path)
            copyfile(label_path, yolov5_labels_train_dir + label_name)
    else: # test dataset
        if os.path.exists(annotation_path):
            test_file.write(image_path + '\n')
            convert_annotation(nameWithoutExtention) # convert label
            copyfile(image_path, yolov5_images_test_dir + voc_path)
            copyfile(label_path, yolov5_labels_test_dir + label_name)
train_file.close()
test_file.close()
print('ok')

Probability: 84
ok


In [10]:
{i:j for i,j in enumerate(names.keys())}

{0: '6',
 1: '7',
 2: '8',
 3: '9',
 4: '10',
 5: '25',
 6: '41',
 7: '105',
 8: '110',
 9: '115',
 10: '148',
 11: '156',
 12: '222',
 13: '228',
 14: '235',
 15: '256',
 16: '280',
 17: '310',
 18: '387',
 19: '392',
 20: '394',
 21: '398',
 22: '401',
 23: '402',
 24: '430',
 25: '480',
 26: '485',
 27: '673'}

In [ ]:
# from xml.dom.minidom import Document
# import os
# import cv2
# def makexml(picPath, txtPath, xmlPath):  # txt所在文件夹路径，xml文件保存路径，图片所在文件夹路径
#     """此函数用于将yolo格式txt标注文件转换为voc格式xml标注文件
#     在自己的标注图片文件夹下建三个子文件夹，分别命名为picture、txt、xml
#     """
#     dic = {
#          0: '6',
#          1: '7',
#          2: '8',
#          3: '9',
#          4: '10',
#          5: '25',
#          6: '41',
#          7: '105',
#          8: '110',
#          9: '115',
#          10: '148',
#          11: '156',
#          12: '222',
#          13: '228',
#          14: '235',
#          15: '256',
#          16: '280',
#          17: '310',
#          18: '387',
#          19: '392',
#          20: '394',
#          21: '398',
#          22: '401',
#          23: '402',
#          24: '430',
#          25: '480',
#          26: '485',
#          27: '673'}
#     files = os.listdir(txtPath)
#     for i, name in enumerate(files):
#         xmlBuilder = Document()
#         annotation = xmlBuilder.createElement("annotation")  # 创建annotation标签
#         xmlBuilder.appendChild(annotation)
#         txtFile = open(txtPath + name)
#         txtList = txtFile.readlines()
#         img = cv2.imread(picPath + name[0:-4] + ".jpg")
#         Pheight, Pwidth, Pdepth = img.shape
 
#         folder = xmlBuilder.createElement("folder")  # folder标签
#         foldercontent = xmlBuilder.createTextNode("driving_annotation_dataset")
#         folder.appendChild(foldercontent)
#         annotation.appendChild(folder)  # folder标签结束
 
#         filename = xmlBuilder.createElement("filename")  # filename标签
#         filenamecontent = xmlBuilder.createTextNode(name[0:-4] + ".jpg")
#         filename.appendChild(filenamecontent)
#         annotation.appendChild(filename)  # filename标签结束
 
#         size = xmlBuilder.createElement("size")  # size标签
#         width = xmlBuilder.createElement("width")  # size子标签width
#         widthcontent = xmlBuilder.createTextNode(str(Pwidth))
#         width.appendChild(widthcontent)
#         size.appendChild(width)  # size子标签width结束
 
#         height = xmlBuilder.createElement("height")  # size子标签height
#         heightcontent = xmlBuilder.createTextNode(str(Pheight))
#         height.appendChild(heightcontent)
#         size.appendChild(height)  # size子标签height结束
 
#         depth = xmlBuilder.createElement("depth")  # size子标签depth
#         depthcontent = xmlBuilder.createTextNode(str(Pdepth))
#         depth.appendChild(depthcontent)
#         size.appendChild(depth)  # size子标签depth结束
 
#         annotation.appendChild(size)  # size标签结束
 
#         for j in txtList:
#             oneline = j.strip().split(" ")
#             object = xmlBuilder.createElement("object")  # object 标签
#             picname = xmlBuilder.createElement("name")  # name标签
#             namecontent = xmlBuilder.createTextNode(dic[oneline[0]])
#             picname.appendChild(namecontent)
#             object.appendChild(picname)  # name标签结束
 
#             pose = xmlBuilder.createElement("pose")  # pose标签
#             posecontent = xmlBuilder.createTextNode("Unspecified")
#             pose.appendChild(posecontent)
#             object.appendChild(pose)  # pose标签结束
 
#             truncated = xmlBuilder.createElement("truncated")  # truncated标签
#             truncatedContent = xmlBuilder.createTextNode("0")
#             truncated.appendChild(truncatedContent)
#             object.appendChild(truncated)  # truncated标签结束
 
#             difficult = xmlBuilder.createElement("difficult")  # difficult标签
#             difficultcontent = xmlBuilder.createTextNode("0")
#             difficult.appendChild(difficultcontent)
#             object.appendChild(difficult)  # difficult标签结束
 
#             bndbox = xmlBuilder.createElement("bndbox")  # bndbox标签
#             xmin = xmlBuilder.createElement("xmin")  # xmin标签
#             mathData = int(((float(oneline[1])) * Pwidth + 1) - (float(oneline[3])) * 0.5 * Pwidth)
#             xminContent = xmlBuilder.createTextNode(str(mathData))
#             xmin.appendChild(xminContent)
#             bndbox.appendChild(xmin)  # xmin标签结束
 
#             ymin = xmlBuilder.createElement("ymin")  # ymin标签
#             mathData = int(((float(oneline[2])) * Pheight + 1) - (float(oneline[4])) * 0.5 * Pheight)
#             yminContent = xmlBuilder.createTextNode(str(mathData))
#             ymin.appendChild(yminContent)
#             bndbox.appendChild(ymin)  # ymin标签结束
 
#             xmax = xmlBuilder.createElement("xmax")  # xmax标签
#             mathData = int(((float(oneline[1])) * Pwidth + 1) + (float(oneline[3])) * 0.5 * Pwidth)
#             xmaxContent = xmlBuilder.createTextNode(str(mathData))
#             xmax.appendChild(xmaxContent)
#             bndbox.appendChild(xmax)  # xmax标签结束
 
#             ymax = xmlBuilder.createElement("ymax")  # ymax标签
#             mathData = int(((float(oneline[2])) * Pheight + 1) + (float(oneline[4])) * 0.5 * Pheight)
#             ymaxContent = xmlBuilder.createTextNode(str(mathData))
#             ymax.appendChild(ymaxContent)
#             bndbox.appendChild(ymax)  # ymax标签结束
 
#             object.appendChild(bndbox)  # bndbox标签结束
 
#             annotation.appendChild(object)  # object标签结束
 
#         f = open(xmlPath + name[0:-4] + ".xml", 'w')
#         xmlBuilder.writexml(f, indent='\t', newl='\n', addindent='\t', encoding='utf-8')
#         f.close()
 
#  if __name__ == "__main__":
#     picPath = "VOCdevkit/VOC2007/JPEGImages/"  # 图片所在文件夹路径，后面的/一定要带上
#     txtPath = "VOCdevkit/VOC2007/YOLOLabels/"  # txt所在文件夹路径，后面的/一定要带上
#     xmlPath = "VOCdevkit/VOC2007/Annotations/"  # xml文件保存路径，后面的/一定要带上
#     makexml(picPath, txtPath, xmlPath)
 

In [13]:
len(names.keys())

28